In [1]:
%run './Dynamic Model.ipynb'

# Hardware

[This](https://www.analog.com/media/en/technical-documentation/data-sheets/ADIS16405.pdf) is the data sheet for our IMU system.

# Covariance Analysis of TRIAD algorithm

Reference 'Fundamentals of Spacecraft Attitude Determination and Control', or alternatively 'Attitude Determination Using Vector Observations: A Fast Optimal Matrix Algorithm'. We are not using the FOMA, but the covariance analysis in it is applicable.

In [2]:
# This right here is the covariance matrix for an attitude estimate from the TRIAD method
# b_1 and b_2 are star pointing vectors in body frame
# sig_1 and sig_2 are sums of error variances of body frame and inertial reference vectors
# this calculation comes straight from weighted_triad() or fast_quaternion() in beast.h
# so this method in python is now unneccessary
def P_TRIAD(b_1, b_2, sig_1, sig_2):
    b1_x_b2 = np.cross(b_1, b_2)
    norm = np.linalg.norm(b1_x_b2)
    b_x = b1_x_b2 / norm
    top = sig_2 * np.outer(b_1, b_1) + sig_1 * np.outer(b_2, b_2)
    bot = norm**2
    side = sig_1 * np.outer(b_x, b_x)
    return top / bot + side

# Multiplicative Extended Kalman Filter

Reference 'Attitude Error Representations for Kalman Filtering'.

Be aware that the attitude and error covariance from star tracker are likely inertial reference, rather than body referenced

In [6]:
# attitude error representation, as quaternion, given gibbs
# we don't end up using this, since we can directly reset
#def delta_q(a_g):
#    front = 1 / np.sqrt(4 + sum([x**2 for x in a_g]))
#    return front * np.array(2, a_g[0], a_g[1], a_g[2])

# change in linear EKF wrt state estimate
# w is estimate not actual
def F_mtrx(w):
    return np.array([
        [0,     w[2],  -w[1], -1, 0, 0],
        [-w[2], 0,     w[0],  0, -1, 0],
        [w[1],  -w[0], 0,     0, 0, -1],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]
    ])

# process noise model, this is a hack. GOTTA TUNE IT
# we assume for now that all variance is independent
def Q_mtrx(gyro_sigma, drift_sigma):
    g = gyro_sigma**2
    d = drift_sigma**2
    return np.array([
        [g, 0, 0, 0, 0, 0],
        [0, g, 0, 0, 0, 0],
        [0, 0, g, 0, 0, 0],
        [0, 0, 0, d, 0, 0],
        [0, 0, 0, 0, d, 0],
        [0, 0, 0, 0, 0, d]
    ])

# gibbs vector attitude representation, singular at 180 degrees
# we can use MRPs instead to avoid sqrts if we need
# we use this representation for numerical niceness
def gibbs(q):
    return 2*q[1:]/q[0]

# kalman gain matrix
def gain(P_a, P_cT, R):
    return np.array([P_a[0],
                     P_a[1],
                     P_a[2],
                     P_cT[0],
                     P_cT[1],
                     P_cT[2]]).dot(np.linalg.inv(P_a + R))

# convenience function
def extract(P):
    top = np.vsplit(P, 2)[0]
    P_a, P_c = np.hsplit(top, 2)
    return P_a, P_c

# change in gyro bias estimate is assumed noise driven
def db_dt(sigma):
    return np.array([np.random.normal(0, sigma)
                     for i in range(3)])

# this seems ugly and i don't know if it's proper. for bias expectation
def b_est(w_out, w_ref):
    return w_out - w_ref

# farrenkopf's gyro dynamics error model
# we ignore output noise for rate-integrating gyros
def w_est(w_out, x, sigma): # takes output from actual gyros, and gyro drift vector
    b = x[3:]
    eta1 = db_dt(sigma)
    return w_out - b - eta1

# linearization of exact kinematic equation for gibbs propagation
def da_g_dt(x, w_ref, w_out, sigma):
    a_g = x[:3]
    b = x[3:]
    eta1 = db_dt(sigma)
    b_hat = b_est(w_out, w_ref)
    return b_hat - b - eta1 - np.cross(w_ref, a_g)

# Riccati equation
# propagation of kalman covariance matrix
def dP_dt(P, F, G, Q): 
    return F.dot(P) + P.dot(F.T) + G.dot(Q.dot(G.T))

# covariance update
def cov_update(P, K):
    P_a, P_c = extract(P)
    return P - K.dot(np.hstack((P_a, P_c)))

# state update, nice and simple
def state_update(x_prior, K, a_obs):
    a_prior = x_prior[:3]
    x_poster = x_prior + K.dot(a_obs - a_prior)
    return x_poster
                     
# after kalman update, reset everything! returns quaternion
# we don't necessarily have to reset after every update
# but we really goddamn should or else
def reset(q_ref, x):
    a_g = x[:3]
    unnorm = product(q_ref, np.array([2, a_g[0], a_g[1], a_g[2]]))
    return unnorm / np.linalg.norm(unnorm)

# interface with reality
def real_world(duration, dt, q_kalman=None, w_kalman=None, true_state=None):
    if true_state != None:
        x, v, q, w, whl, cur, time = true_state
        full_state = master(duration, dt, x, v, q, w, whl, cur, time,
                            'continuous', 'holdpose', q_kalman, w_kalman)
    else:
        full_state = master(duration)
    x, v, q, w, whl, t, env, cur, mags, time = full_state
    x, v, q, w, whl, cur, time = x[-1], v[-1], q[-1], w[-1], whl[-1], cur[-1], time
    true_state = (x, v, q, w, whl, cur, time)
    return true_state

def measure(true_state):
    x, v, q, w, whl, cur, time = true_state
    return q, w

# a priori state estimation
def propagation(w_out, x, P,
                gyro_sigma, drift_sigma,
                G, Q,
                dt, length):
    t=0
    # integrate propagation over length of update step
    while t < length:
        w_ref = w_est(w_out, x, gyro_sigma)
        F = F_mtrx(w_ref)
        dadt = da_g_dt(x, w_ref, w_out, gyro_sigma)
        dbdt = db_dt(drift_sigma)
        dPdt = dP_dt(P, F, G, Q)
        # euler method till i'm less lazy and runge kutta it
        x += dt * np.array([dadt, dbdt]).flatten()
        P += dt * dPdt
        t += dt
    return x, P, w_ref
        
def kalman_step(x, P, K, t,
                q_obs, R, w_out,
                q_ref, gyro_sigma, drift_sigma,
                G, Q,
                dt_e, dt_k):        
    # initialize progagation integration
    x_prior = np.copy(x[-1])
    P_prior = np.copy(P[-1])
    # Kalman Gain
    P_a, P_c = extract(P_prior)
    K.append(gain(P_a, P_c.T, R))
        
    # propagate
    x_prior, P_prior, w_ref = propagation(w_out, x_prior, P_prior,
                                gyro_sigma, drift_sigma,
                                G, Q,
                                dt_e, dt_k)
        
    # update kalman state to next step
    a_obs = gibbs(error_quat(q_obs, q_ref)) # q_ref is a priori, q_obs comes from startracker
    P.append(cov_update(P_prior, K[-1]))
    x.append(state_update(x_prior, K[-1], a_obs))
    t.append(t[-1] + dt_k)
    q_ref = reset(q_ref, x[-1]) # reset q_ref
    return x, P, K, t, q_ref, w_ref
 
def kalman_init(gyro_sigma, drift_sigma, sensor_sigma, bias_sigma, t_end):
    # Initialize
    dt_k = 1. # kalman update timestep
    dt_e = 0.5 # propagation timestep
    dt_s = 0.5 # simulation timestep
    
    # change in linear EKF with respect to noise
    G = np.array([
    [-1, 0, 0, 0, 0, 0],
    [0, -1, 0, 0, 0, 0],
    [0, 0, -1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1.]
    ])
    Q = Q_mtrx(gyro_sigma, drift_sigma) # process noise
    R = np.diag(np.array([sensor_sigma**2 for i in range(3)])) # measurement covariance
    
    true_state = real_world(dt_s*2, dt_s)
    q_obs, w_out = measure(true_state)
    P_0 = np.copy(Q)
    x_0 = np.array([np.zeros(3), db_dt(bias_sigma)]).flatten()
    q_ref = np.copy(q_obs)
    w_ref = np.copy(w_out)
    
    x = [x_0]
    P = [P_0]
    true = [true_state]
    P_a, P_c = extract(P_0)
    K = [gain(P_a, P_c.T, R)]
    t = [0]
    
    # simulation length
    while t[-1] < t_end:
        true.append(real_world(dt_k, dt_s, q_ref, w_ref, true[-1]))
        q_obs, w_out = measure(true[-1])
        x, P, K, t, q_ref, w_ref = kalman_step(x, P, K, t,
                            q_obs, R, w_out,
                            q_ref, gyro_sigma, drift_sigma,
                            G, Q,
                            dt_e, dt_k)
    return x, P, K, t, true, q_ref, w_ref

def testit():
    bias_sigma = 0.05236 # rad/s initial bias std dev
    gyro_sigma = 5.818 * 10**(-4) # angular random walk, rad/sqrt(s), from datasheet
    drift_sigma = 1.713 * 10**(-8) # rate random walk, rad / s^(3/2), arbitrary guess from paper
    sensor_sigma = 1.5 * 10**(-5) # rad, star tracker measurement noise, arbitrary guess from paper
    
    x, P, K, t, true, q_ref, w_ref = kalman_init(gyro_sigma, drift_sigma, sensor_sigma, bias_sigma, 60)
    q_obs, w_out = measure(true[-1])
    print("Prediction: \nAttitude:", q_ref, '\nAngular Rate:', w_ref)
    print("\nSimulation: \nAttitude:", q_obs, '\nAngular Rate:', w_out)
    
testit()

Prediction: 
Attitude: [-0.07436497  0.32745614 -0.20094589  0.92025164] 
Angular Rate: [ 0.10234964 -0.0080992  -0.07152028]

Simulation: 
Attitude: [-0.07409991  0.32779975 -0.20065397  0.92021438] 
Angular Rate: [ 0.15216638 -0.03238124  0.00128231]
